## Baseline

Start with simple data pipeline and make a crappy classifier. This will just the simplest thing that works.

Eliminate all unlabeled profiles, using 80/20 train/test split without consideration for
singleton labels or good stratification of samples, using just composition words as features:

Naive Bayes (default config) 54%/45% train/test

Linear SVM (default config) 96%/51% 

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import numpy as np
import profile_classification as pc
raw=pc.load_raw_profile_data()
labeled=pc.get_labeled_profiles(raw)
data=pc.make_train_test_data(labeled)
#print [(k,v[:10]) for k,v in data.items()]

In [55]:
from mcc_transformers import ItemSelector,EPAText
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion


X_train,y_train=data['X_train'],data['y_train']
X_test,y_test=data['X_test'],data['y_test']


pl=Pipeline([('union',
             FeatureUnion(transformer_list=[('composition',
                                           Pipeline([('get_comp',ItemSelector('Composition')),
                                                    ('vectorize',CountVectorizer(min_df=1,ngram_range=(1,1)))])),
                                           ('epaCodes',
                                           Pipeline([('get_codes',ItemSelector('EPA Codes')),
                                                    ('vectorize',EPAText())]))]))])



#vect=CountVectorizer(min_df=1,ngram_range=(1,1))
#svd=TruncatedSVD()
X_v_train=pl.fit_transform(X_train)
#X_v_train=pl.transform(X_train)
X_v_test=pl.transform(X_test)

mNB=MultinomialNB()
svc=LinearSVC()
dt=DecisionTreeClassifier(min_samples_leaf=1)
print pl.named_steps['union'].get_feature_names()[:10]
for clf in (mNB,svc,dt,): 
    clf.fit(X_v_train,y_train)
    y_pred=clf.predict(X_v_test)
    print np.sum(y_pred==y_test)/float(len(y_test))
    #print classification_report(y_test,y_pred)
    

AttributeError: Transformer composition (type Pipeline) does not provide get_feature_names.

In [54]:
r_w=raw.dropna(subset=['Composition'])
r_w=r_w['Composition']
vect.fit(r_w)


NameError: name 'vect' is not defined

In [101]:
print len(vect.get_feature_names())

9258
